### This is a notebook to take narrowband images with the camera (but manually controlling the AT spectrograph instrument externally)

In [1]:
# import libraries of CSCs
import SALPY_calibrationElectrometer as Electrometer
import SALPY_atMonochromator as Monochromator
import SALPY_sedSpectrometer as FiberSpectrograph
import SALPY_atcamera as ATcamera
import SALPY_atArchiver as ATArchiver

In [2]:
# import dependencies
import salobj
import nest_asyncio
nest_asyncio.apply() # fix the tornado5 asyncio issue with notebooks
import asyncio
import time
import os
#import posixpath
#import aiohttp # $ pip install aiohttp
#from contextlib import closing
#from astropy.io import fits
#from matplotlib import pyplot as plt

# try:
#     from urlparse import urlsplit
#     from urllib import unquote
# except ImportError: # Python 3
#     from urllib.parse import urlsplit, unquote
    
%matplotlib inline

/home/saluser/lsst_repos/salobj/python/salobj/test_utils.py:37: UserWarning: Could not import SALPY_Test; TestComponent will not work
  warnings.warn("Could not import SALPY_Test; TestComponent will not work")


In [3]:
import logging
logging.getLogger().setLevel(logging.INFO)
logging.basicConfig(level=logging.INFO,
                format='[%(asctime)s] [%(name)-12s:%(lineno)-4d] [%(levelname)-8s]: %(message)s',
                datefmt='%m-%d %H:%M:%S')

In [56]:
# Define a class with the script that performs the desired task
class NarrowBandAuxTelImages:
    
    """
    Initialize the class and define the components we want to interact with
    """
    def __init__(self):
        """
        Setup each CSC to be an object that can be used to send commands
        as well as receive events/telemetry
        object name follows PEP8, but f-string uses XML name (same
        as import statement above)
        """
        logging.info('Inside __init__')
        self.fiber_spectrograph = salobj.Remote(FiberSpectrograph, f"sedSpectrometer")
        self.electrometer = salobj.Remote(Electrometer, f"calibrationElectrometer:1")
        self.monochromator = salobj.Remote(Monochromator, f"atMonochromator")
        self.at_camera = salobj.Remote(ATcamera, f"atcamera")
        self.at_archiver = salobj.Remote(ATArchiver, f"atArchiver")
        logging.info('End of __init__')
        
    async def takeSequence(self, wavelength_arr, exp_time_arr,
                       grating_arr, n_exps, **kwargs):
        """
        This function defines the sequence of events that need to be performed
        
        Expected input:
        wavelength_arr - single integer or array of wavelengths to loop over (float)
        exp_time_arr - single integer or array of exposure times for each wavelength (float)
        grating_arr - single integer or array of grating values to be used for each wavelength
        n_exps - single integer or array of integers dictating how many files per setup
        
        Optional inputs:
        entry_slit_width: -- not implemented
        exit_slit_width: -- not implemented
        
        """ 
        logging.info('Starting sequence')
        
        # Temporarily hard-code the slit widths
        entry_slit_width = 1.0 # [mm]
        exit_slit_width = 1.0 # [mm]
        # Verify inputs are sensible
        # check exposure time is greater than 1 second
        if all(i >= 1 for i in exp_time_arr) == False:
            raise ValueError('exp_time_arr has values less than 1 second')
        
        # make n_exps and grating_arr arrays if they're single values
        if len(wavelength_arr) > 1:
            if len(grating_arr) == 1:
                grating_arr = [grating_arr]*len(wavelength_arr)
            if len(exp_time_arr) == 1:
                exp_time_arr = [exp_time_arr]*len(wavelength_arr)
            
        # Check that gratings and wavelengths are within ranges
        # just hard code the values until we have config files to check against
        if all( (i == 1 or i == 2) for i in grating_arr) == False:
            raise ValueError('grating_arr has values not equal to 1 or 2')
        
        if all( (300 < i < 1100) for i in wavelength_arr) == False:
            raise ValueError('wavelength_arr has values either less 300 or greater than 1100 nm')
        
        # start loop over wavelengths
        for i, wave in enumerate(wavelength_arr):
            logging.info('Wavelength {0:1.0f} loop starting. Wavelength {1:1.0f} of {2:1.0f}'.format(
            wave,i,len(wavelength_arr)))
            # setup monochromator
            monochromator_cmd_obj = self.monochromator.cmd_updateMonochromatorSetup.DataType()
            monochromator_cmd_obj.gratingType=grating_arr[i]
            monochromator_cmd_obj.wavelength=wave
            monochromator_cmd_obj.fontExitSlitWidth=exit_slit_width
            monochromator_cmd_obj.fontEntranceSlitWidth=entry_slit_width
            # sends command and waits for completion before continuing
            logging.info('Setting up Monochromator')
            #await self.monochromator.cmd_updateMonochromatorSetup.start(monochromator_cmd_obj)
            logging.info('Monochromator in position')
            
            # start loop over wavelengths
            for n in n_exps:
                logging.info('Starting wavelength loop {} of {} for wavelength {}'.format(n, n_exps,wave))
                # start electrometer exposure 
                electrometer_cmd_obj = self.electrometer.cmd_startScan.DataType()
                # send command but don't wait for completion
                #task = loop.create_task(self.electrometer.cmd_startScan.start(electrometer_cmd_obj))
                #logging.info('Starting Electrometer Scanning'.format(n, n_exps,wave))
                #loop.run_until_complete(task)
                
                #self.electrometer.evt_detailedState.flush()
                tmp = self.electrometer.evt_detailedState.next(flush=True, timeout=10)
                self.electrometer.cmd_startScan.start(electrometer_cmd_obj)
                logging.info('Grabbing detailed state')
                detailed_state = await tmp
                # wait for event saying it started (changed state) then wait X seconds
                #task2 = self.electrometer.evt_detailedState.next(flush=True, timeout=5)
                logging.info('Checking for detailed state, got {}'.format(detailed_state.detailedState))
                #detailed_state = loop.run_until_complete(task2)
                
                #task_group = await asyncio.gather(task1,task2)
                
                if detailed_state.detailedState != 8:
                    raise ValueError('Expected Electrometer detailed state of X but got {}'
                                     .format(detailed_state.detailedState))
                
                #print(detailed_state)
                # sleep for 1 second to get ambient light measurement
                wait_time=1.0
                await asyncio.sleep(wait_time)
                
                # Now want to start fiber_spectrograph and at_camera to expose at the same time
                fiber_spectrograph_cmd_obj = self.fiber_spectrograph.cmd_captureSpectImage.DataType()
                fiber_spectrograph_cmd_obj.imageType = 'FLAT_Narrowband'
                fiber_spectrograph_cmd_obj.integrationTime = exp_time_arr[i]
                fiber_spectrograph_cmd_obj.lamp = 'QH_Kiloarc'
                fiber_spectrograph_task = self.fiber_spectrograph.cmd_captureSpectImage.start(fiber_spectrograph_cmd_obj)

                # start camera exposure
                at_camera_cmd_obj = self.at_camera.cmd_takeImages.DataType()
                at_camera_cmd_obj.numImages = 1 # hardcoded since this doesn't work yet
                at_camera_cmd_obj.shutter = True # Means we can't take biases
                at_camera_cmd_obj.expTime = exp_time_arr[i]
                at_camera_cmd_obj.imageSequenceName = 'asdfasdf'
                at_camera_cmd_obj.science = True
                at_camera_task = self.at_camera.cmd_takeImages.start(at_camera_cmd_obj)
                
                # wait for completion of both tasks
                logging.info('Starting at_camera and fiber_spectrograph exposures')
                task_group = await asyncio.gather(fiber_spectrograph_task, at_camera_task)
                logging.info('at_camera and fiber_spectrograph exposures complete')
                # wait for ambient light measurement then stop the electrometer 
                await asyncio.sleep(wait_time)
            
                electrometer_cmd_obj = self.electrometer.cmd_stopScan.DataType()
                logging.info('stopping electrometer')
                await self.electrometer.cmd_stopScan.start(electrometer_cmd_obj)
                logging.info('electrometer stopped')
        

In [57]:
wavelength_arr=[600]
exp_time_arr=[1.0]
grating_arr=[1]
n_exps=[1.0]

seq = NarrowBandAuxTelImages()

[10-24 17:42:24] [root        :14  ] [INFO    ]: Inside __init__
[10-24 17:42:25] [root        :20  ] [INFO    ]: End of __init__


In [58]:
loop = asyncio.get_event_loop()

In [60]:
#print('Start')
loop.run_until_complete(seq.takeSequence(wavelength_arr, 
                                         exp_time_arr, 
                                         grating_arr, 
                                         n_exps))
#print('Done')

[10-24 17:43:19] [root        :38  ] [INFO    ]: Starting sequence
[10-24 17:43:19] [root        :66  ] [INFO    ]: Wavelength 600 loop starting. Wavelength 0 of 1
[10-24 17:43:19] [root        :74  ] [INFO    ]: Setting up Monochromator
[10-24 17:43:19] [root        :76  ] [INFO    ]: Monochromator in position
[10-24 17:43:19] [root        :80  ] [INFO    ]: Starting wavelength loop 1.0 of [1.0] for wavelength 600
/home/saluser/stack/python/miniconda3-4.3.21/lib/python3.6/site-packages/ipykernel_launcher.py:90: RuntimeWarning: coroutine 'RemoteCommand._wait_done' was never awaited
[10-24 17:43:20] [root        :92  ] [INFO    ]: Grabbing detailed state
[10-24 17:43:20] [root        :96  ] [INFO    ]: Checking for detailed state, got 8
[10-24 17:43:21] [root        :127 ] [INFO    ]: Starting at_camera and fiber_spectrograph exposures
[10-24 17:43:24] [root        :129 ] [INFO    ]: at_camera and fiber_spectrograph exposures complete
[10-24 17:43:25] [root        :134 ] [INFO    ]: sto

In [42]:
tmp=seq.electrometer.evt_detailedState.get()
tmp.detailedState

8

In [43]:
"""
This is used to bring the individual CSCs into the enabled state.
This will be handled by the OCS in the future.
This assumes CSCs are in the disabled state
"""

        start_topic_fiberSpectrograph = self.fiberSpectrometer.cmd_start.DataType()
        start_topic_fiberSpectrograph.settingsToApply = 'Default'

        start_topic_electrometer = self.electrometer.cmd_start.DataType()
        start_topic_electrometer.settingsToApply = 'Default'
        
        start_topic_electrometer = self.electrometer.cmd_start.DataType()
        start_topic_electrometer.settingsToApply = 'Default'

        task_start_sed = self.sedSpectrometer.cmd_start.start(start_topic_sed)
        task_start_cel = self.calibrationElectrometer.cmd_start.start(start_topic_cel)
    


IndentationError: unexpected indent (<ipython-input-43-96f820779c94>, line 7)

In [6]:
wavelength_arr=[3.3,3,5,3.2]
grating_arr=[1,2,1,1,2]
exp_time_arr=[1.2,2.2,1.2,1.3,5]
try:
    test1 = len(wavelength_arr)
    test2 = len(grating_arr)
    test3 = len(exp_time_arr)
    test = isinstance(wavelength_arr,(float))
    print(test)
    
except NameError as err:
    raise NameError("ERROR: {} variable is not defined: ".format(err))
    
if ~isinstance(wavelength_arr,(int,float)):
    raise ValueError(f"wavelength_arr must contain only integers or floats")
if ~isinstance(grating_arr,(int,float)):
    raise ValueError(f"grating_arr must be only integer values")
if ~isinstance(exp_time_arr,(int,float)):
    raise ValueError(f"exp_time_arr must only integers or floats")

False


ValueError: wavelength_arr must contain only integers or floats

In [13]:
wavelength_arr[:] is float

False

In [53]:
[1] *3

[1, 1, 1]

In [30]:
if all(exp_time_arr[i] > 1 for i,val in enumerate(exp_time_arr)) == True:
            raise ValueError('exp_time_arr has values less than 1 second')

In [31]:
exp_time_arr=[1.0]
for i in range(len(exp_time_arr)):
    print(i)

0
